In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("./data/salary/train.csv")

In [2]:
#change obj column to num
def to_num(df):
    col_name = df.columns.values
    obj_list = []
    for name in col_name:
        if df[name].dtype == 'object':
            obj_list.append(name)
    obj_dict = {}
    for name in obj_list:#list of object column
        keys = list(dict(list(df.groupby(name))).keys())#various value of this column
        obj_dict[name] = keys
        for i in range(len(keys)):
            key = keys[i]
            df_col = df[name]
            df_col[df[name] == key] = i
    return df,obj_dict
# change num converted from obj to one-hot
def sperate_obj(df):
    col_name = df.columns.values
    obj_list = []
    for name in col_name:
        if df[name].dtype == 'object':
            obj_list.append(name)
    obj_dict = {}
    for name in obj_list:#list of object column
        keys = list(dict(list(df.groupby(name))).keys())#various value of this column
        
        if len(keys) <= 2 :#value types no more than 2,change previous column only
            obj_dict[name] = keys
            df_col = df[name]
            df_col[df[name] == keys[0]] = 0
            df_col[df[name] != 0] = 1
        else:#need convert to on-hot,i.e.need add and drop columns
            obj_dict[name] = []
            for key in keys:
                #add new col
                col_name = name+"_"+key
                df[col_name] = 0
                new_col = df[col_name]
                new_col[df[name] == key] = 1
                obj_dict[name].append(col_name)
                
            #drop col
            df.drop(name,axis = 1,inplace=True)
        
    return df,obj_dict

In [3]:
df_convert,obj_dict = sperate_obj(df)

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
df_convert.to_csv("./result/salary/test.csv")

In [ ]:
f = open("./result/salary/test",'w')
f.writelines(str(obj_dict))
f.close()

In [5]:
df_conv_0 = df_convert[df_convert['income'] == 0].drop('income',axis=1)
df_conv_1 = df_convert[df_convert['income'] == 1].drop('income',axis=1)

In [6]:
data_x_0 = df_conv_0.values.astype("float").T#(dim,n)
data_x_1 = df_conv_1.values.astype("float").T
#no need feature scaling

In [7]:
n_0 = data_x_0.shape[1]
n_1 = data_x_1.shape[1]
n = n_0 + n_1

mu_0 = np.mean(data_x_0,axis=1).reshape((-1,1))#(dim,1)
mu_1 = np.mean(data_x_1,axis=1).reshape((-1,1))

sigma_0 = np.cov(data_x_0)#(dim,dim)
sigma_1 = np.cov(data_x_1)
sigma = (float(n_0)/float(n)) * sigma_0 + (float(n_1)/float(n)) * sigma_1
sigma_inv = 1 / (sigma)

In [8]:
w = np.matmul((mu_1-mu_0).T,sigma_inv).T
b = -0.5 * np.matmul(np.matmul(mu_1.T,sigma_inv),mu_1) \
    + 0.5 * np.matmul(np.matmul(mu_0.T,sigma_inv),mu_0) \
    + np.log(float(n_1)/float(n_0))

In [12]:
def sigmoid(x):
    return 1./(1.+np.exp(-x))
def f(x,w,b):
    z = np.matmul(w.T,x) + b
    pred = sigmoid(z)
    return pred

def loss(x,y,w,b):
    pred = f(x,w,b)
    pred_log = np.log(pred)#q(1) = f(x)
    no_pred_log = np.log(1.0-pred)#q(0) = 1 - f(x) ; f(x) : probability of class A(1)
    loss_val = -1.* (np.matmul(y,pred_log.T) + np.matmul((1.0-y),no_pred_log.T))
    return loss_val

def accuarcy(x,y,w,b):
    pred = f(x,w,b)
    pred = np.rint(pred)
    return np.mean(np.equal(pred,y).astype('float'))

In [10]:
df_label = df_convert.loc[:,'income']
df_train = df_convert.drop(labels=['income'],axis=1)
data_y = df_label.values.reshape((1,-1)).astype(np.float)
#must use astype(float),otherwise the dtype is object,will err in numpy operation
data_x = df_train.values.astype(np.float) .T

In [11]:
pred_train = f(data_x,w,b)

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [14]:
accuarcy(data_x,data_y,w,b)

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


0.5693928319154817

In [15]:
def load_test(df,obj_dict):
    col_name = df.columns.values
    obj_list = []
    for name in col_name:
        if df[name].dtype == 'object':
            obj_list.append(name)
   
    for name in obj_list:#list of object column
        keys = list(obj_dict[name])#various value of this column
        
        if len(keys) <= 2 :#value types no more than 2,change previous column only;no prefix
            df_col = df[name]
            df_col[df[name] == keys[0]] = 0
            df_col[df[name] != 0] = 1
        else:#need convert to on-hot,i.e.need add and drop columns
            keys = [key[len(name)+1:] for key in keys]#has prefix
            for key in keys:
                #add new col
                col_name = name+"_"+key
                df[col_name] = 0
                new_col = df[col_name]
                new_col[df[name] == key] = 1
                obj_dict[name].append(col_name)
                
            #drop col
            df.drop(name,axis = 1,inplace=True)
        
    return df

df_t = pd.read_csv("./data/salary/test.csv",encoding='Big5')
df_t_c = load_test(df_t,obj_dict)

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
df_t_c.to_csv("./result/salary/temp.csv")

In [16]:
test_x = df_t_c.values.astype('float').T
#no feature scaling

pred = np.rint(f(test_x,w,b))#<=50K : 0; >50K : 1

pred_list = list(pred.reshape((-1,)))
id_list = range(1,len(pred_list)+1)
df_out = pd.DataFrame({'id':id_list,'label':pred_list},dtype='int')

C:\Users\a2323\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [17]:
df_out.to_csv("./result/salary/pred.csv",index=False)
